<a href="https://colab.research.google.com/github/subharoy2000/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-/blob/main/Copy_of_Palisade_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright (C) 2021 Textualization Software Ltd. Distributed under the terms of the [Apache Software License 2.0](http://www.apache.org/licenses/LICENSE-2.0).
---

<a href="https://colab.research.google.com/github/subharoy2000/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-/blob/main/notebooks/Palisade_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In [ ]:
!rm -rf encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-
!git clone https://github.com/subharoy2000/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-

Cloning into 'encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 177 (delta 53), reused 57 (delta 33), pack-reused 89
Receiving objects: 100% (177/177), 6.36 MiB | 20.46 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [ ]:
!cp /content/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-/dependencies/palisade-install.tar.gz /content; cd /content; tar -xzf palisade-install.tar.gz
!cd /content/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-/data; tar -xzf synthetic-invoices.tar.gz

In [ ]:
!mkdir -p /content/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-/build/lib
!cd /content/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-; c++ -fopenmp -fPIC -std=gnu++11 `python3.7-config --cflags` `python3.7-config --includes` -I/content/palisade-install/include/palisade{,/core,/pke,/third-party/inclue} -o build/lib/ckks_wrapper.o -c src/ckks_wrapper.cpp 
!cd /content/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-; c++ -fopenmp -fPIC -std=gnu++11 `python3.7-config --cflags` `python3.7-config --includes` -I/content/palisade-install/include/palisade{,/core,/pke,/third-party/inclue} -o build/lib/pycrypto.o -c src/pycrypto.cpp 
!cd /content/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-; /usr/bin/c++ -fPIC  -Wall -Werror -O3  `python3.7-config --ldflags` -DPALISADE_VERSION=1.11.4 -Wno-parentheses -fopenmp -shared -Wl,-soname,pycrypto.so.1 -lpython3.7m -o build/lib/pycrypto.so.1 build/lib/ckks_wrapper.o build/lib/pycrypto.o   -L/content/palisade-install/lib  -Wl,-rpath,/content/palisade-install/lib:  /content/palisade-install/lib/libPALISADEcore.so.1 /content/palisade-install/lib/libPALISADEbinfhe.so.1 /content/palisade-install/lib/libPALISADEpke.so.1  /usr/lib/x86_64-linux-gnu/libpython3.7m.so /usr/lib/x86_64-linux-gnu/libboost_python3-py36.so 
!cd /content/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-/build/lib; ln -s pycrypto.so.1 pycrypto.so

In [ ]:
import sys
sys.path.append('/content/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-/build/lib')

import pycrypto

In [ ]:
import os
import os.path
import random
import math

import xml.etree.ElementTree as ET

import numpy as np
import timeit

In [ ]:
import xml.etree.ElementTree as ET
path = '/content/encrypted_machine_learning_on_Heart_Disease_Dataset-HDD-/data_new/invoices_new'
files = os.listdir(path)
print(len(files))

34


In [ ]:
# from sklearn import tree
# root = ET.getroot()
# print(root.tag, root.attrib)
# for child in root:
#   print(child.tag, child.attrib)

In [ ]:
# print(ET.tostring(root, encoding='utf16').decode('utf16'))

In [ ]:
# print(ET.tostring(root, encoding='utf8').decode('utf8'))

In [ ]:
DATA_DIR='/content/encrypted_machine_learning_on_Heart_Disease_Dataset/data_new/invoices_new'

matrix           = list() # rows = [ rfc, pref dict id->count ]
Enc_Packets          = dict() # rfc -> [ row in matrix, name ]
clients_by_row   = list() # row -> [ rfc, name ]
Systems         = dict() # id -> [ col in matrix, name ]
products_by_col  = list() # col -> [ id, name ]

for invoice in os.listdir(DATA_DIR):
    if '.xml' not in invoice:
        continue
    tree = ET.parse(os.path.join(DATA_DIR, invoice))
    root = tree.getroot()
    texture_se = root[0]
    _id=root[1]
    if texture_se not in Enc_Packets:
        # new row
        # FOR PRESENTATION
        # if len(matrix) > 400:
        #  break
        Enc_Packets[texture_se] = [ len(matrix), root[1] ]  #.attrib['radius_mean']
        matrix.append( [ texture_se, dict() ] )
        clients_by_row.append( [ (texture_se), root[2] ] )  #.attrib['radius_mean']
    row = Enc_Packets[texture_se][0]

    for concept in root[2]:
        _id = concept.attrib['radius_mean']
        name = concept.attrib['symmetry_mean']

        if _id not in Systems:
            # new col
            Systems[_id] = [ len(Systems), root[1] ]
            products_by_col.append( [ _id, root[0] ])
        col = Systems[_id][0]
        matrix[row][1][col] = matrix[row][1].get(col, 0) + 1

print("Loaded ", len(Enc_Packets), " Encrypted Packets ")

FileNotFoundError: ignored

In [ ]:
import csv
m = np.zeros((len(Enc_Packets), len(Systems)))

for r in range(len(matrix)):
    for c, v in matrix[r][1].items():
        m[r][c] = v

random.seed(21656456)

chosen = random.choice(list(Enc_Packets.keys()))
chosen_row = Enc_Packets[chosen][0]
#chosen = list(clients.keys())[67]
print("Recommendations for", chosen, Enc_Packets[chosen][1])

products_for_client = list(matrix[Enc_Packets[chosen][0]][1].items())
random.shuffle(products_for_client)
print("Sample from", len(products_for_client),"Decryption halted")
for pair in products_for_client[:20]:
    print("\t", products_by_col[pair[0]][1], 'Decrypted', pair[1], 'times')

# similarity to each client using raw inner product as we cannot take sqrt

client_simil = np.ndarray((len(Enc_Packets),1))

minus_row = -1 * m[chosen_row]

for idx in range(len(Enc_Packets)):
    #client_simil[idx] = math.sqrt(np.dot(m[idx], m[chosen_row]))
    diff = m[idx] + minus_row
    client_simil[idx] = np.sum(diff*diff)

print("Top De-similarities:")
print(client_simil[:10])

recos = np.zeros((len(Systems),))

for idx in range(len(Enc_Packets)):
    weight = client_simil[idx]
    row = m[idx][:]
    recos = recos + row * weight
    #if idx % 100 == 0:
    #  print("{} {:,} {:,} {:,}".format(idx, recos[0], recos[idx], recos[8200]))

recos_indexed = sorted(list(map(lambda p: [ p[1], p[0] ], enumerate(recos))))

printed = 0
for score, col in reversed(recos_indexed):
    if m[chosen_row][col] > 0:
        continue
    print("\t({}) {} score={:,}".format(col, products_by_col[col][1], score))
    printed += 1
    if printed > 10:
        break
old_recos = recos
old_recos_indexed = recos_indexed

Recommendations for <Element 'row' at 0x7fbad7125710> <Element 'row' at 0x7fbad7125350>
Sample from 0 Decryption halted
Top De-similarities:
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [ ]:
def next_power_of_2(x):
    return 1 if x == 0 else 2**(x - 1).bit_length()

# CKKS related parameters
# min_depth=1
max_depth=4
scale_factor=30
batch_size=next_power_of_2(max(len(Systems), len(Enc_Packets)) // 2 + 2)

print("batch size:", batch_size)

print("Initializing ckks wrapper")
crypto=pycrypto.CKKSwrapper()
print("Initialized wrapper")

print("Generating keys")
start_time = timeit.default_timer()
crypto.KeyGen(max_depth, scale_factor, batch_size)
print("Keys generated in", timeit.default_timer() - start_time)

print("Generating key Batches")
start_time = timeit.default_timer()
crypto.KeyGen(max_depth, scale_factor, batch_size)
print("Keys generated in", timeit.default_timer() - start_time)

print("Encrypting")
start_time = timeit.default_timer()
m = list()

for r in range(len(matrix)):
    row = [ 0.0 for _ in range(len(Systems)) ]
    for c, v in matrix[r][1].items():
        row[c] =  v * 1.0
    m.append( [ crypto.Encrypt( row[:batch_size] ), crypto.Encrypt( row[batch_size:] ) ] )

print("Encrypted in", timeit.default_timer() - start_time)

random.seed(210826)

# use chosen from before
#chosen = random.choice(list(clients.keys()))
#chosen_row = clients[chosen][0]

# similarity to each client using raw inner product as we cannot take sqrt

print("Negating row")
start_time = timeit.default_timer()
minus_row = [ crypto.EvalMultConst(m[chosen_row][0], [ -1.0 for _ in range(batch_size) ]),
              crypto.EvalMultConst(m[chosen_row][1], [ -1.0 for _ in range(len(Systems) - batch_size) ]) ]
print("Minus row in", timeit.default_timer() - start_time)

print("Asserting row")
start_time = timeit.default_timer()
minus_row = [ crypto.EvalMultConst(m[chosen_row][0], [ -1.0 for _ in range(batch_size) ]),
              crypto.EvalMultConst(m[chosen_row][1], [ -1.0 for _ in range(len(Systems) - batch_size) ]) ]
print("Plus row in", timeit.default_timer() + start_time)

batch size: 32
Initializing ckks wrapper
Initialized wrapper
Generating keys
Keys generated in 1.1603132329999823
Generating key Batches
Keys generated in 1.1246105510000177
Encrypting
Encrypted in 2.172726938000011
Negating row
Minus row in 0.021455836000001227
Asserting row
Plus row in 319.214405743
